<a href="https://colab.research.google.com/github/JCbarajas5/Machine-Learning-Anahuac/blob/main/Practica2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1.  Cargue la base de datos Machines.csv a un DataFrame

In [ ]:
import pandas as pd

df = pd.read_csv("Machines.csv")
display(df.head())
print("Shape:", df.shape)

,RPM,Load_pct,Temp_C,Vibration_RMS,Pressure_kPa,Current_A,Acoustic_dB,OilParticles_cnt,Failure
0,1826.2,52.93,29.51,1.648,254.58,9.982,67.42,84.2,0
1,1490.0,83.49,35.36,1.966,285.32,12.390,67.38,73.8,0
2,1937.6,26.79,33.01,1.458,225.16,9.139,63.28,37.0,1
3,1985.1,64.44,28.34,1.706,256.85,12.471,64.91,90.1,0
4,1262.2,84.16,32.97,2.499,300.12,12.514,65.51,93.5,1


Shape: (600, 9)


##2. Calcule la proporción de cada clase de la variable Failure y responda: ¿el conjunto de datos está balanceado o desbalanceado?


In [ ]:
proporciones = df["Failure"].value_counts(normalize=True).sort_index()
conteos = df["Failure"].value_counts().sort_index()

display(pd.DataFrame({"conteo": conteos, "proporción": proporciones}))

p_min = proporciones.min()
if p_min < 0.4:
    print("Conclusión: el conjunto está DESBALANCEADO.")
else:
    print("Conclusión: el conjunto está BALANCEADO.")

,conteo,proporción
Failure,,
0,408,0.68
1,192,0.32


Conclusión: el conjunto está DESBALANCEADO.


##3. Defina variables predictoras (todas menos Failure) y variable objetivo (Failure).


In [ ]:
X = df.drop(columns=["Failure"])
y = df["Failure"]

print("Predictoras (X):", list(X.columns))
print("Objetivo (y): Failure")

Predictoras (X): ['RPM', 'Load_pct', 'Temp_C', 'Vibration_RMS', 'Pressure_kPa', 'Current_A', 'Acoustic_dB', 'OilParticles_cnt']
Objetivo (y): Failure


##4. Divida su dataset en conjunto de entrenamiento (80%) y de prueba (20%) y estratifique de acuerdo a variable objetivo Y ( use random_state=42)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(" X_train:", X_train.shape, "X_test:", X_test.shape)
print("Proporciones en train:", y_train.value_counts(normalize=True).to_dict())
print("Proporciones en test :", y_test.value_counts(normalize=True).to_dict())

 X_train: (480, 8) X_test: (120, 8)
Proporciones en train: {0: 0.6791666666666667, 1: 0.32083333333333336}
Proporciones en test : {0: 0.6833333333333333, 1: 0.31666666666666665}


##5. Estandarice variables predictoras


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc  = scaler.transform(X_test)

print("Media (train, aprox 0):", X_train_sc.mean(axis=0).round(3))
print("Std  (train, aprox 1):", X_train_sc.std(axis=0).round(3))

Media (train, aprox 0): [-0.  0.  0.  0.  0.  0. -0. -0.]
Std  (train, aprox 1): [1. 1. 1. 1. 1. 1. 1. 1.]


##6. Genere los modelos de máquinas de vectores de soporte y árboles de decisión.


In [ ]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

svm_model = SVC(kernel="rbf", C=1.0, gamma="scale")  # baseline razonable
tree_model = DecisionTreeClassifier(random_state=42)  # baseline

svm_model.fit(X_train_sc, y_train)
tree_model.fit(X_train_sc, y_train)

DecisionTreeClassifier(random_state=42)


##7. Obtenga métricas para cada uno de ellos y matriz de confusión


In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score
)
import numpy as np

def evaluar_modelo(nombre, model, Xte, yte, usa_decision_function=False):
    y_pred = model.predict(Xte)
    cm = confusion_matrix(yte, y_pred)  # orden: [[TN, FP],[FN, TP]] si clases 0/1
    out = {
        "modelo": nombre,
        "accuracy": accuracy_score(yte, y_pred),
        "precision": precision_score(yte, y_pred, zero_division=0),
        "recall": recall_score(yte, y_pred),
        "f1": f1_score(yte, y_pred),
        "TN": cm[0,0], "FP": cm[0,1], "FN": cm[1,0], "TP": cm[1,1],
    }
    # ROC AUC (opcional pero útil)
    if usa_decision_function:
        scores = model.decision_function(Xte)
        out["roc_auc"] = roc_auc_score(yte, scores)
    else:
        proba = model.predict_proba(Xte)[:,1] if hasattr(model, "predict_proba") else None
        out["roc_auc"] = roc_auc_score(yte, proba) if proba is not None else np.nan

    print(f"\n=== {nombre} ===")
    print("Matriz de confusión :\n", cm)
    print("\nReporte de clasificación:\n\n", classification_report(yte, y_pred, digits=4))
    return out

resultados = []
resultados.append(evaluar_modelo("SVM (RBF)", svm_model, X_test_sc, y_test, usa_decision_function=True))
resultados.append(evaluar_modelo("Decision Tree", tree_model, X_test_sc, y_test, usa_decision_function=False))

res_df = pd.DataFrame(resultados).set_index("modelo")
display(res_df[["accuracy","precision","recall","f1","roc_auc","TN","FP","FN","TP"]].round(4))


=== SVM (RBF) ===
Matriz de confusión :
 [[78  4]
 [ 6 32]]

Reporte de clasificación:

               precision    recall  f1-score   support

           0     0.9286    0.9512    0.9398        82
           1     0.8889    0.8421    0.8649        38

    accuracy                         0.9167       120
   macro avg     0.9087    0.8967    0.9023       120
weighted avg     0.9160    0.9167    0.9160       120


=== Decision Tree ===
Matriz de confusión :
 [[75  7]
 [ 8 30]]

Reporte de clasificación:

               precision    recall  f1-score   support

           0     0.9036    0.9146    0.9091        82
           1     0.8108    0.7895    0.8000        38

    accuracy                         0.8750       120
   macro avg     0.8572    0.8521    0.8545       120
weighted avg     0.8742    0.8750    0.8745       120



,accuracy,precision,recall,f1,roc_auc,TN,FP,FN,TP
modelo,,,,,,,,,
SVM (RBF),0.9167,0.8889,0.8421,0.8649,0.9730,78,4,6,32
Decision Tree,0.8750,0.8108,0.7895,0.8000,0.8521,75,7,8,30



##8. Basándose en todas las métricas, indique que modelo considera mejor y porqué


In [ ]:

mejor_por_f1 = res_df["f1"].idxmax()
mejor_por_auc = res_df["roc_auc"].idxmax()

print("Mejor por F1 :", mejor_por_f1, "-> F1 =", float(res_df.loc[mejor_por_f1, "f1"]))
print("Mejor por AUC:", mejor_por_auc, "-> AUC =", float(res_df.loc[mejor_por_auc, "roc_auc"]))

if mejor_por_f1 == mejor_por_auc:
    print(f"Conclusión: {mejor_por_f1} es el mejor modelo global (mejor F1 y mejor AUC).")
else:
    print("Conclusión: depende de la métrica priorizada. Si priorizas F1 el mejor es:", mejor_por_f1,
          "y si priorizas AUC el mejor es:", mejor_por_auc)

# SVM suele generalizar mejor que un árbol sin poda/tuning en datos continuos estandarizados.

Mejor por F1 : SVM (RBF) -> F1 = 0.8648648648648649
Mejor por AUC: SVM (RBF) -> AUC = 0.9730423620025673
Conclusión: SVM (RBF) es el mejor modelo global (mejor F1 y mejor AUC).


##9. Desde el punto de vista del mantenimiento de máquinas: ¿Qué métrica considera más importante?


In [ ]:
for modelo in res_df.index:
    recall = float(res_df.loc[modelo, "recall"])
    fn = int(res_df.loc[modelo, "FN"])
    print(f"{modelo}: Recall(Failure=1) = {recall:.4f} | FN = {fn}")

print("la métrica más importante suele ser el RECALL de Failure=1 (minimiza falsos negativos).")

SVM (RBF): Recall(Failure=1) = 0.8421 | FN = 6
Decision Tree: Recall(Failure=1) = 0.7895 | FN = 8
la métrica más importante suele ser el RECALL de Failure=1 (minimiza falsos negativos).


##10. Desde el punto de vista de mantenimiento de máquinas ¿qué tipo de error es más grave: falso positivo o falso negativo? Justifique su respuesta.

In [ ]:
best = mejor_por_f1
fp = int(res_df.loc[best, "FP"])
fn = int(res_df.loc[best, "FN"])

print("Modelo ilustrativo:", best)
print("FP (falsos positivos):", fp, "-> alarma de falla cuando no falla")
print("FN (falsos negativos):", fn, "-> NO detectas una falla real")

print("""\n en mantenimiento suele ser más grave el FALSO NEGATIVO (FN).
Justificación: predices 'no falla' pero sí falla -> paros no planificados, daño a equipos, riesgos de seguridad y costos altos.
Un FP normalmente solo implica una inspección/mantenimiento extra (costo menor y controlable).""")

Modelo ilustrativo: SVM (RBF)
FP (falsos positivos): 4 -> alarma de falla cuando no falla
FN (falsos negativos): 6 -> NO detectas una falla real

 en mantenimiento suele ser más grave el FALSO NEGATIVO (FN).
Justificación: predices 'no falla' pero sí falla -> paros no planificados, daño a equipos, riesgos de seguridad y costos altos.
Un FP normalmente solo implica una inspección/mantenimiento extra (costo menor y controlable).
